In [35]:
import pandas as pd
import numpy as np
import pickle
import sklearn
from sklearn.model_selection import train_test_split

In [36]:
import import_ipynb
import utility
import Model
import EWC

In [37]:
from keras import backend as K
from utility import LoadDataNoDefCW_EWC
from EWC import evaluate, compute_precision_matrices, compute_elastic_penalty, ewc_loss, train_with_ewc
from Model import DFNet
import random
from keras.utils import np_utils
from keras.optimizers import Adamax
import numpy as np
import os

random.seed(0)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [38]:
# Use only CPU
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

description = "Training and evaluating DF model for closed-world scenario on non-defended dataset"

print(description)
# Training the DF model
NB_EPOCH = 30   # Number of training epoch
print ("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 128 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 10000 # Packet sequence length
OPTIMIZER = Adamax(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0) # Optimizer

NB_CLASSES = 95 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)

Training and evaluating DF model for closed-world scenario on non-defended dataset
Number of Epoch:  30


In [39]:
# Data: shuffled and split between train and test sets
print ("Loading and preparing data for training, and evaluating the model")
X_A_train, X_B_train, y_A_train, y_B_train, X_A_test, X_B_test, y_A_test, y_B_test = LoadDataNoDefCW_EWC()
# Please refer to the dataset format in readme
K.set_image_data_format("channels_last") # tf is tensorflow

Loading and preparing data for training, and evaluating the model
Loading non-defended dataset for closed-world scenario
Data dimensions:
X_A: Training data's shape :  (6020,)
X_B: Training data's shape :  (7280,)
y_A: Training data's shape :  (6020,)
y_B: Training data's shape :  (7280,)
X_A: Testing data's shape :  (2580,)
X_B: Testing data's shape :  (3120,)
y_A: Testing data's shape :  (2580,)
y_B: Testing data's shape :  (3120,)


In [40]:
# Building and training model
print ("Building and training DF model")

model = DFNet.build(input_shape=INPUT_SHAPE, classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
	metrics=["accuracy"])
print ("Model compiled")

Building and training DF model
Model compiled


In [41]:
# Function to adjust the sequence length to a target length
def adjust_sequence_length(sequence, target_length=10000, padding_value=-1):
    if len(sequence) < target_length:
        # If the sequence is shorter than the target length, pad with the specified value
        sequence = sequence + [padding_value] * (target_length - len(sequence))
    else:
        # If the sequence is longer than the target length, truncate it
        sequence = sequence[:target_length]
    return sequence

# Adjust sequence lengths for each dataset
X_A_train = np.array([adjust_sequence_length(seq) for seq in X_A_train])
X_B_train = np.array([adjust_sequence_length(seq) for seq in X_B_train])
X_A_test = np.array([adjust_sequence_length(seq) for seq in X_A_test])
X_B_test = np.array([adjust_sequence_length(seq) for seq in X_B_test])

# Convert data to float32 type
X_A_train = X_A_train.astype('float32')
X_A_test = X_A_test.astype('float32')
X_B_train = X_B_train.astype('float32')
X_B_test = X_B_test.astype('float32')


In [42]:
# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_A_train = X_A_train[:, :,np.newaxis]
X_A_test = X_A_test[:, :,np.newaxis]
X_B_train = X_B_train[:, :,np.newaxis]
X_B_test = X_B_test[:, :,np.newaxis]

print(X_A_train.shape[0], 'train samples')
print(X_A_test.shape[0], 'test samples')
print(X_B_train.shape[0], 'train samples')
print(X_B_test.shape[0], 'test samples')

6020 train samples
2580 test samples
7280 train samples
3120 test samples


In [43]:
# Convert class vectors to categorical classes matrices
y_A_train = np_utils.to_categorical(y_A_train, NB_CLASSES)
y_A_test = np_utils.to_categorical(y_A_test, NB_CLASSES)
y_B_train = np_utils.to_categorical(y_B_train, NB_CLASSES)
y_B_test = np_utils.to_categorical(y_B_test, NB_CLASSES)

In [44]:
y_A_train.shape

(6020, 95)

In [45]:
train_with_ewc(DFNet, X_A_train, X_A_test, y_A_train, y_A_test, X_B_train, X_B_test, y_B_train, y_B_test, epochs=100)

Building and training DF model
Model compiled
Epoch 1/100
48/48 - 11s - loss: 3.6654 - accuracy: 0.1120 - 11s/epoch - 221ms/step
Epoch 2/100
48/48 - 8s - loss: 2.9130 - accuracy: 0.2367 - 8s/epoch - 173ms/step
Epoch 3/100
48/48 - 8s - loss: 2.5670 - accuracy: 0.3056 - 8s/epoch - 174ms/step
Epoch 4/100
48/48 - 8s - loss: 2.3277 - accuracy: 0.3696 - 8s/epoch - 174ms/step
Epoch 5/100
48/48 - 8s - loss: 2.1280 - accuracy: 0.4161 - 8s/epoch - 175ms/step
Epoch 6/100
48/48 - 8s - loss: 1.9576 - accuracy: 0.4635 - 8s/epoch - 175ms/step
Epoch 7/100
48/48 - 8s - loss: 1.8062 - accuracy: 0.5066 - 8s/epoch - 175ms/step
Epoch 8/100
48/48 - 8s - loss: 1.6624 - accuracy: 0.5512 - 8s/epoch - 175ms/step
Epoch 9/100
48/48 - 8s - loss: 1.5599 - accuracy: 0.5836 - 8s/epoch - 176ms/step
Epoch 10/100
48/48 - 8s - loss: 1.4532 - accuracy: 0.6068 - 8s/epoch - 176ms/step
Epoch 11/100
48/48 - 8s - loss: 1.3915 - accuracy: 0.6201 - 8s/epoch - 176ms/step
Epoch 12/100
48/48 - 8s - loss: 1.3024 - accuracy: 0.6530 -